<a href="https://colab.research.google.com/github/tanvinaminul/ML-DL-IITKGP/blob/master/Naive_Bayes_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from sklearn import datasets

iris = datasets.load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

df["target"] = iris.target

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("iris.csv")
df = df.drop("Id", axis = 1)
display(df)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
train= df.sample(frac=0.7, random_state=1)
test= df. drop(train.index)

y_train= train["Species"]
x_train= train.drop('Species', axis= 1)

y_test= test["Species"]
x_test= test.drop('Species', axis= 1)


In [ ]:
means = train.groupby(["Species"]).mean() 
display(means)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,
Iris-setosa,5.084848,3.493939,1.457576,0.248485
Iris-versicolor,5.925714,2.785714,4.268571,1.325714
Iris-virginica,6.718919,2.989189,5.632432,2.056757


In [ ]:
var = train.groupby(["Species"]).var()
display(var)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,
Iris-setosa,0.132576,0.176212,0.024394,0.010076
Iris-versicolor,0.234319,0.085378,0.196924,0.033143
Iris-virginica,0.366577,0.116547,0.337808,0.060856


In [ ]:
prior = (train.groupby("Species").count()/ len(train)).iloc[:,1]
display(prior)

Species
Iris-setosa        0.314286
Iris-versicolor    0.333333
Iris-virginica     0.352381
Name: SepalWidthCm, dtype: float64

In [ ]:
classes = np.unique(train["Species"].tolist()) 
print(classes)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [ ]:
def Normal(n, mu, var):
    
    # Function to return pdf of Normal(mu, var) evaluated at x
    sd = np.sqrt(var)
    pdf = (np.e ** (-0.5 * ((n - mu)/sd) ** 2)) / (sd * np.sqrt(2 * np.pi))
    
    return pdf

def Predict(X):
    Predictions = []
    
    for i in X.index: # Loop through each instances
        
        ClassLikelihood = []
        instance = X.loc[i]
        
        for cls in classes: # Loop through each class
            
            FeatureLikelihoods = []
            FeatureLikelihoods.append(np.log(prior[cls])) # Append log prior of class 'cls'
            
            for col in x_train.columns: # Loop through each feature
                
                data = instance[col]
                
                mean = means[col].loc[cls] # Find the mean of column 'col' that are in class 'cls'
                variance = var[col].loc[cls] # Find the variance of column 'col' that are in class 'cls'
                
                Likelihood = Normal(data, mean, variance)
                
                if Likelihood != 0:
                    Likelihood = np.log(Likelihood) # Find the log-likelihood evaluated at x
                else:
                    Likelihood = 1/len(train) 
                
                FeatureLikelihoods.append(Likelihood)
                
            TotalLikelihood = sum(FeatureLikelihoods) # Calculate posterior
            ClassLikelihood.append(TotalLikelihood)
            
        MaxIndex = ClassLikelihood.index(max(ClassLikelihood)) # Find largest posterior position
        Prediction = classes[MaxIndex]
        Predictions.append(Prediction)
        
    return Predictions

In [ ]:
PredictTrain = Predict(x_train)
PredictTest = Predict(x_test)

In [ ]:
def Accuracy(y, prediction):
    
    # Function to calculate accuracy
    y = list(y)
    prediction = list(prediction)
    score = 0
    
    for i, j in zip(y, prediction):
        if i == j:
            score += 1
            
    return score / len(y)

In [ ]:
round(Accuracy(y_train, PredictTrain), 5)

0.98095

In [ ]:
round(Accuracy(y_test, PredictTest), 5)

0.91111